In [1]:
# pip install the libraries
import time
import pickle
import torch
import skimage.io as io
from PIL import Image
# huggingface
from transformers import CLIPProcessor, CLIPModel

In [2]:
captions_dir = '/kaggle/input/2017-2017/annotations_trainval2017/annotations/captions_train2017.json'
img_dir = '/kaggle/input/2017-2017/train2017/train2017'

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
import json

with open(captions_dir,'r') as f:
    data = json.load(f) 

In [5]:
x = data['annotations']
for el in x:
    print(el['image_id'])
    print(el['caption'])
    break

del x

203564
A bicycle replica with a clock as the front wheel.


In [6]:
# loading the CLIP model
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
preprocess = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
filename = '/kaggle/input/2017-2017/train2017/train2017/000000000009.jpg'
image = io.imread(filename)                       # converts image to numpy array
pil_img = Image.fromarray(image)                  # convert to PIL image    
processed_img = preprocess(images = pil_img, return_tensors='pt', padding=True)   # preprocess the image
image_tensor = processed_img['pixel_values']
image = image_tensor.to(device)

In [10]:
with torch.no_grad():
    out = model.get_image_features(pixel_values = image)

In [11]:
out

tensor([[-1.9665e-01,  4.2181e-01,  2.7624e-02,  5.1787e-01,  2.9352e-01,
         -2.1178e-01,  1.5872e-01, -8.9810e-02,  4.2665e-02, -6.5934e-03,
          2.8311e-01,  1.4174e-01, -8.6217e-02,  1.1674e-01,  3.1542e-01,
          2.6665e-01,  7.2710e-01,  1.7859e-01,  1.2847e-01, -6.1216e-01,
         -2.0868e+00,  5.3938e-02, -6.0309e-02, -3.1067e-01,  3.8256e-01,
          2.0433e-01, -1.5418e-01,  2.9678e-02, -1.7846e-01, -6.5795e-02,
         -1.2425e-01,  2.0375e-01, -3.9768e-01, -2.7158e-01,  3.4267e-02,
         -3.9222e-01, -2.3287e-01, -4.0635e-01, -2.5672e-01,  2.5351e-01,
         -1.3027e-01, -6.8080e-02,  2.1023e-01,  2.7543e-01, -1.8051e-01,
         -1.3398e+00,  2.8315e-01,  4.5093e-01, -6.1926e-02,  8.7234e-03,
          1.1537e-01, -5.8120e-01,  7.0603e-02, -2.2887e-01,  5.6195e-02,
          2.8087e-01,  3.1550e-01, -1.7536e-01,  1.2494e-01, -2.3966e-01,
          2.9839e-01,  1.8632e-01, -1.1953e-01,  8.6714e-02, -2.9076e-01,
          5.2507e-01, -1.2599e-02,  1.

In [12]:
out.size()

torch.Size([1, 512])

In [13]:
img_files_loc = '/kaggle/input/2017-2017/train2017/train2017/'

def get_img_loc(img_id):
    img_id = str(img_id)
    padded_id = img_id.rjust(12,'0')
    filename = img_files_loc + padded_id + '.jpg'
    return filename


In [14]:
get_img_loc(9)

'/kaggle/input/2017-2017/train2017/train2017/000000000009.jpg'

In [15]:
out_path = '/kaggle/working/'

In [16]:
len(data['annotations'])

591753

In [18]:
# 3-4 hours

In [ ]:
# parsing code
image_captions = []
prefix = []                # clip outputs
ct = 0
start = time.time()

for el in data['annotations']:
    
    img_id = el['image_id']
    caption = el['caption']
    ct += 1
    
    filename = get_img_loc(img_id)
    image = io.imread(filename)                       # converts image to numpy array
    pil_img = Image.fromarray(image)                  # convert to PIL image    
    processed_img = preprocess(images = pil_img, return_tensors='pt', padding=True)        # preprocess the image
    image_tensor = processed_img['pixel_values']
    image = image_tensor.to(device)

    with torch.no_grad():
        out = model.get_image_features(pixel_values = image)

    image_captions.append(caption)
    prefix.append(out)

    if ct % 50000 == 0 :             # saving for safety purposes
        print('Checkpoint: ',ct)
        with open(out_path + str(ct) + '.pkl', 'wb') as f:
            pickle.dump({"clip_embedding": torch.cat(prefix, dim=0), "captions": image_captions}, f)


end = time.time()
print((end-start)/60)
with open(out_path + 'final_embeddings' + '.pkl', 'wb') as f:
    pickle.dump({"clip_embedding": torch.cat(prefix, dim=0), "captions": image_captions}, f)

Checkpoint:  50000
